In [ ]:
# Measurment of Demographic Parity in Machine Learning Predictions

In [ ]:
!pip install fairlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 52.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, demographic_parity_ratio, demographic_parity_difference, selection_rate
import seaborn as sns

In [ ]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
# Step 2: Data preprocessing

# dropping unnecessary columns
df.drop(columns=['class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone'], inplace=True)

# dropping rows with missing values
df.dropna(inplace=True)

# Label Encoding the sex and embarked columns
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])
df['embarked'] = le.fit_transform(df['embarked'])

# Creating feature and target variables
X = df.drop(columns=['survived'])
y = df['survived']

# train test split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# using standard scaler for feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print("Data preprocessing done successfully")

Data preprocessing done successfully


In [ ]:
# Train the Logistic Regression model

# Initialize the model
model = LogisticRegres

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# get the metrics
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)



Accuracy:  0.7990654205607477


In [ ]:
# Measure Demographic Parity

# Sensitive feature: sex
sensitive_feature = df.loc[y_test.index, 'sex']

# Create metric frame to compute fairness and keep accuracy and selection rate
metric_frame = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "selection_rate": selection_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

dp_differ = demographic_parity_difference(y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_feature)
dp_ratio = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_feature)

print("Demographic Parity Difference: ", dp_differ)
print("Demographic Parity Ratio: ", dp_ratio)

print(metric_frame.by_group)


Demographic Parity Difference:  0.7825386292539577
Demographic Parity Ratio:  0.10066456040963069
     accuracy  selection_rate
sex                          
0    0.857143        0.870130
1    0.766423        0.087591
